# File there

In [208]:
test_Library = !(find -name "Library_sample")
test_peak = [x.replace("Library_sample","RESULTS.CSV") for x in test_Library]

In [209]:
def preakInfo(peak_path) :
    Peak_list = []
    f = open(peak_path,"r")
    a = f.read()
    f.close()
    spl_1 = '''Header=,"Peak","R.T.","First","Max","Last","PK  TY","Height","Area","Pct Max","Pct Total"\r\n'''
    spl_2 = '''Header=,"PK","RT","Area Pct","Library/ID","Ref","CAS","Qual"\r\n''' 
    b = a.split(spl_1)[1].split(spl_2)
    ori_PK_dat = b[0].split("\r\n")[:-3]
    for ori in ori_PK_dat :
        d = ori.split(",")
        Peak = int(d[1])
        RT = d[2]
        scan_num = int(d[4])
        intensity = d[8]
        Peak_list.append([Peak,RT,scan_num,intensity])
    return Peak_list


def OnePeakCAS(bam) :
    # bam is string with one peak data
    # bam == b[i]
    PeakCAS = []
    CAS_item = {}
    for j in bam.split("\r\n") :
        k = j.strip()
        if "W9N08.L" in k :
            L = k.split()
            peak = int(L[0])
            Area = float(L[2])
        elif len(k) > 34 :
            k_set = filter(None, k.split(" "))
            #print k_set
            CAS,Qual = k_set[-2] ,float(k_set[-1])
            CAS_item[CAS] = Qual
    for m in CAS_item.items() :
        PeakCAS.append([peak,Area]+list(m))
    return PeakCAS

def compInfo(one_Lib) :
    Comp_list = []
    X = []
    f = open(one_Lib,"r")
    a = f.read().split("_____________________________________________________________________________\r\n")
    f.close()
    b = a[1].split(' \r\n')[:-2]
    for i in b :
        if "W9N08.L" in i :
            X.append(i)
    for i in X :
        Comp_list.append(OnePeakCAS(i))
    return Comp_list




def Comp_Peak_join(compinfo,pkinfo) :
    JoinedPeak = []
    for index , comp_data in enumerate(compinfo) :
        for W in comp_data :
            JoinedPeak.append( pkinfo[index]+ W[1:] )
    return JoinedPeak
# Peak , RT , scan number , intensity, Area% , CAS-ID , Qual

In [225]:
# Peak_final = [['Class', 'Sample', 'Date', "Peak", '  RT', "scan number", "Intensity", "Area%", 'CAS-ID', "Qual"]]
Peak_final = []
for one_peak in test_peak :
    path_info = one_peak.split("/")
    one_Lib = one_peak.replace("RESULTS.CSV","Library_sample")
    
    if path_info[-3] == "Y" :
        ## This is for sample Y (all rotten)
        sample_class = path_info[-3]
        sample_info = path_info[-2].replace(".D","")
        sample = sample_info[4:]
        date_info = sample_info[:4]
        mon,day = date_info[:2],date_info[2:]
        date = "2015/"+mon+"/"+day
        
        basic_list = [sample_class,sample, date]
        # print basic_list
        
        compinfo = compInfo(one_Lib)
        pkinfo = preakInfo(one_peak)
        
        
        for pk in Comp_Peak_join(compinfo,pkinfo) :
            Peak_final.append(basic_list + pk)
    else : 
        ## This for normal sample
        sample_class = path_info[-4]
        sample_info = path_info[-2].replace(".D","")
        sample = sample_info[4:]
        date_info = sample_info[:4]
        mon,day = date_info[:2],date_info[2:]
        date = "2015/"+mon+"/"+day
        
        basic_list = [sample_class,sample, date]
        compinfo = compInfo(one_Lib)
        pkinfo = preakInfo(one_peak)
        
        #print basic_list
        
        for pk in Comp_Peak_join(compinfo,pkinfo) :
            Peak_final.append(basic_list + pk)


# Dataframe

In [230]:
import pandas as pd

df = pd.DataFrame(Peak_final)
# 'Class', 'Sample', 'Date', "Peak", '  RT', "scan number", "Intensity", "Area%", 'CAS-ID', "Qual"
df = df.rename(columns={0: 'Class', 1: 'Sample',2:"Date",3: "Peak",4:"RT", 5: "scan number",6 :"Intensity",\
                        7 : "Area%", 8 : "CAS_ID", 9 : "Quality"})
print df.shape , df.size
print "Number of CAS-ID : ",len(df.CAS_ID.unique())

df.head()

(23819, 10) 238190
Number of CAS-ID :  3061


,Class,Sample,Date,Peak,RT,scan number,Intensity,Area%,CAS_ID,Quality
0,CU,CU3,2015/11/03,1,1.771,450,104584,0.14,000124-38-9,4.0
1,CU,CU3,2015/11/03,2,1.932,493,103807,0.14,000124-38-9,4.0
2,CU,CU3,2015/11/03,3,7.510,1981,103443,0.14,000563-46-2,35.0
3,CU,CU3,2015/11/03,3,7.510,1981,103443,0.14,000930-18-7,35.0
4,CU,CU3,2015/11/03,3,7.510,1981,103443,0.14,000627-20-3,35.0


In [234]:
df_Qual75 = df[ df['Quality']> 75]
print df_Qual75.shape,df_Qual75.size
print "Reduce rate : ",100 - float(df_Qual75.size)/df.size*100
print "Number of CAS-ID : ",len(df_Qual75.CAS_ID.unique())
df_Qual75.head()

(8468, 10) 84680
Reduce rate :  64.4485494773
Number of CAS-ID :  479


,Class,Sample,Date,Peak,RT,scan number,Intensity,Area%,CAS_ID,Quality
9,CU,CU3,2015/11/03,6,8.203,2166,199396,0.26,000123-72-8,78.0
11,CU,CU3,2015/11/03,8,8.380,2213,49279,0.07,000534-22-5,76.0
12,CU,CU3,2015/11/03,9,8.533,2254,769124,1.02,000109-99-9,90.0
13,CU,CU3,2015/11/03,10,8.593,2270,78929,0.10,000109-99-9,80.0
15,CU,CU3,2015/11/03,12,8.897,2351,133877,0.18,000071-43-2,91.0


In [235]:
df_Qual90 = df[ df['Quality']> 90]
print df_Qual90.shape,df_Qual90.size
print "Reduce rate : ",100 - float(df_Qual90.size)/df.size*100
print "Number of CAS-ID : ",len(df_Qual90.CAS_ID.unique())
df_Qual90.head()

(3664, 10) 36640
Reduce rate :  84.6173223057
Number of CAS-ID :  233


,Class,Sample,Date,Peak,RT,scan number,Intensity,Area%,CAS_ID,Quality
15,CU,CU3,2015/11/03,12,8.897,2351,133877,0.18,000071-43-2,91.0
25,CU,CU3,2015/11/03,19,10.348,2738,42397,0.06,024587-26-6,97.0
26,CU,CU3,2015/11/03,19,10.348,2738,42397,0.06,024587-27-7,95.0
27,CU,CU3,2015/11/03,19,10.348,2738,42397,0.06,019264-50-7,95.0
32,CU,CU3,2015/11/03,21,10.617,2810,505338,0.67,000108-88-3,94.0


In [236]:
df.to_csv("full_CAS.csv")
df_Qual75.to_csv("75_CAS.csv")
df_Qual90.to_csv("90_CAS.csv")

http://stackoverflow.com/questions/28017091/will-pandas-dataframe-object-work-with-sklearn-kmeans-clustering 